In [1]:
import os, itertools, subprocess
import numpy as np
import pandas as pd

from mpire import WorkerPool

RAW_PATH = "/data2/zhoujb/project/cowpea_project/rawData/"
RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
CROSS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Cross_Analysis_Results/rawCross/"
CROSS_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Cross_Analysis_Results/VCVLCross/CrossRes/"
VCVL_CROSS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Cross_Analysis_Results/VCVLCross/"

In [4]:
output_file = pd.DataFrame()
file_dir = os.listdir(CROSS_RES_PATH)
for file_name in file_dir:
    tmp_data = pd.read_table(os.path.join(CROSS_RES_PATH, file_name))
    tmp_data = tmp_data[tmp_data["Delta_score"]>0]
    tmp_data = tmp_data.head(10)
    output_file = pd.concat([output_file, tmp_data], axis=0, ignore_index=True)

output_file.to_csv(os.path.join(VCVL_CROSS_PATH, "VCVL_P1_P2_CROSS_TOP10.txt"), sep="\t", index=0)

output_file[["P1", "P2"]].to_csv(os.path.join(VCVL_CROSS_PATH, "VCVL_P1_P2_TOP10.txt"), sep="\t", index=0)

In [5]:
p1_p2_info = pd.read_table(os.path.join(VCVL_CROSS_PATH, "VCVL_P1_P2_CROSS_TOP10.txt"))
p2_info = pd.DataFrame(columns=["P2", "P2_score", "P2_donate_times", 
                                "Optimal_score_mean", "Delta_score_mean", "Same_mean", "Keep_mean", "Replaced_by_P2_mean"])
for group in p1_p2_info.groupby("P2"):
    group_name, group_data = group
    p2_score = group_data["P2_score"].mean()
    p2_donate_times = len(group_data)
    optimal_score_mean = group_data['Optimal_score'].mean()
    delta_score_mean = group_data['Delta_score'].mean()
    same_mean = group_data['Same'].mean()
    keep_mean = group_data['Keep'].mean()
    replaced_by_p2_mean = group_data['Replaced_by_P2'].mean()
    p2_info.loc[len(p2_info)] = [group_name, p2_score, p2_donate_times, 
                                 optimal_score_mean, delta_score_mean, same_mean, keep_mean, replaced_by_p2_mean]
    #print(group_name, len(group_data))

p2_info = p2_info.sort_values(by=["P2_donate_times"], ascending=[False])
p2_info.to_csv(os.path.join(VCVL_CROSS_PATH, "VCVL_TOP10_P2_info.txt"), sep="\t", index=0)

In [6]:
p2_info = pd.read_table(os.path.join(VCVL_CROSS_PATH, "VCVL_TOP10_P2_info.txt"))
for index in p2_info.head(10).index:
    p2_name = p2_info.loc[index, "P2"]
    tmp_file = pd.read_table(os.path.join(CROSS_RES_PATH, "{}.cross.res".format(p2_name)))
    tmp_file.to_excel(os.path.join(VCVL_CROSS_PATH, "VCVLTmpFile/{}_cross_res.xlsx".format(p2_name)), index=0)
    del p2_name, tmp_file